<a href="https://colab.research.google.com/github/will-gun/stock-data/blob/master/collectTaiwanStockData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import time
from io import StringIO
import requests
import pandas as pd
import pickle

def create_today_timestamp():
    today = time.strftime("%Y-%m-%d",time.gmtime())
    return int(time.mktime(time.strptime(today, "%Y-%m-%d")))

def create_timestamp_from_today(n):
    today = create_today_timestamp()
    return today + n*24*3600
tomorrow_timestamp = create_timestamp_from_today(1)

def create_tw_stock_info_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    #res = requests.get("https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=%E4%B8%8A%E5%B8%82&INDUSTRY_CAT=%E4%B8%8A%E5%B8%82%E5%85%A8%E9%83%A8&SHEET=%E4%BA%A4%E6%98%93%E7%8B%80%E6%B3%81&SHEET2=%E6%97%A5&RPT_TIME=%E6%9C%80%E6%96%B0%E8%B3%87%E6%96%99")
    df = pd.read_html(res.text)[0]
    print(df)
    df.columns = df.iloc[0]
    df = df.iloc[2:900]
    df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)
    #df = df.dropna(how='any')
    df = df.reset_index(drop=True)
    print(df)
    new_df = df['有價證券代號及名稱'].str.replace(u'\u3000',' ').str.split(u' ',expand=True)
    print(new_df)
    new_df.columns = ['Ticker', 'StockName']
    new_df['Sector'] = df['產業別']
    return new_df

tw_stock_info_df = create_tw_stock_info_list()
tw_stock_info_df

stock_df = pd.DataFrame()
#ticker_list = tw_stock_info_df['Ticker']
ticker_list = tw_stock_info_df['Ticker'].tail(30)
for ticker in ticker_list:
    print('## Info: Download Ticker '+ticker+'!')
    site = "https://query1.finance.yahoo.com/v7/finance/download/"+ticker+".TW?period1=0&period2="+str(tomorrow_timestamp)+"&interval=1d&events=history&crumb=hP2rOschxO0"
    try:
        response = requests.get(site)
        tmp_df = pd.read_csv(StringIO(response.text))
        tmp_df['Ticker'] = ticker
        stock_df = pd.concat([stock_df,tmp_df],axis=0)
        print(stock_df)
        
    except:
        print('## Warning: Ticker '+ticker+' is failed!')
stock_df = stock_df.reset_index(drop=True)
stock_df = stock_df[['Date','Ticker','Open','High','Low','Close','Adj Close','Volume']]



                   0                    1           2    3     4        5    6
0          有價證券代號及名稱  國際證券辨識號碼(ISIN Code)         上市日  市場別   產業別  CFICode   備註
1                 股票                   股票          股票   股票    股票       股票   股票
2            1101　台泥         TW0001101004  1962/02/09   上市  水泥工業   ESVUFR  NaN
3            1102　亞泥         TW0001102002  1962/06/08   上市  水泥工業   ESVUFR  NaN
4            1103　嘉泥         TW0001103000  1969/11/14   上市  水泥工業   ESVUFR  NaN
...              ...                  ...         ...  ...   ...      ...  ...
21595  01003T　兆豐新光R1         TW00001003T4  2005/12/26   上市   NaN   CBCIXU  NaN
21596  01004T　土銀富邦R2         TW00001004T2  2006/04/13   上市   NaN   CBCIXU  NaN
21597  01007T　兆豐國泰R2         TW00001007T5  2006/10/13   上市   NaN   CBCIXU  NaN
21598  01009T　王道圓滿R1         TW00001009T1  2018/06/21   上市   NaN   CBCIXU  NaN
21599  01010T　京城樂富R1         TW00001010T9  2018/12/05   上市   NaN   CBCIXU  NaN

[21600 rows x 7 columns]
0      有價證券代號及名稱 國際證券辨識號碼(